- 截取视频前一分钟：https://www.bilibili.com/video/BV1wK411v7c1
- 用 https://github.com/cunjian/pytorch_face_landmark 检测并绘制视频
中每帧的 facial landmarks，最后再合成为带 facial landmarks 的新视频。
<br><br>
- 注意：
- a. 存在多张脸时，每帧只画最大的人脸。
- b. 处理视频过程中，所有 weight 应该只加载一次（不能每帧都去加载）。
- c. 结果应该包括完整的可执行代码（包括 github 仓库本身）及视频

In [ ]:
# git clone https://github.com/cunjian/pytorch_face_landmark.git

提取前1分钟视频

In [6]:
from moviepy.video.io.VideoFileClip import VideoFileClip

# 读取视频
video_path = r'I:\mydesk\bishi\data\Video.mp4'
video = VideoFileClip(video_path)

# 截取前1分钟视频
trimmed_video = video.subclip(0, 60)  # 60表示1分钟的秒数

# 保存截取后的视频
output_path = 'trimmed_video.mp4'
trimmed_video.write_videofile(output_path)


Moviepy - Building video trimmed_video.mp4.
MoviePy - Writing audio in trimmed_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video trimmed_video.mp4



Moviepy - Done !
Moviepy - video ready trimmed_video.mp4


合成新视频

In [ ]:
import cv2
import torch
from models import models
from utils import util
from moviepy.editor import VideoFileClip
from tqdm import tqdm

# 加载权重文件
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.FAN(1)
model = util.load_weights(model, 'checkpoint.pth', device=device)
model.to(device)
model.eval()

# 读取视频并获取相关信息
video_path = 'I:\mydesk\bishi\data\trimmed_video.mp4'
output_path = 'output_video.mp4'
video = VideoFileClip(video_path)
fps = video.fps
total_frames = int(video.duration * fps)

# 保存处理后的视频
processed_video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (int(video.size[0]), int(video.size[1])))

# 处理视频每一帧
for i in tqdm(range(total_frames)):
    frame = video.get_frame(i / fps)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # 使用模型检测人脸关键点
    with torch.no_grad():
        image = torch.from_numpy(frame.transpose((2, 0, 1))).float().div(255.0).unsqueeze(0).to(device)
        _, landmarks = model(image)
        landmarks = landmarks[0].cpu().numpy().reshape(-1, 2)

    # 获取最大人脸的关键点并绘制
    face_idx = util.get_largest_face(landmarks)
    landmarks = landmarks[face_idx].astype(int)
    for (x, y) in landmarks:
        cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)

    processed_video.write(frame)

# 释放资源
video.close()
processed_video.release()
